# Задание 1

In [1]:
import os
from bs4 import BeautifulSoup

fns = os.listdir('./1/zip_var_2')

data = []
for i, fn in enumerate(fns):
    with open(f'./1/zip_var_2/{fn}', encoding="utf-8") as f:
        lines = f.read()
    bs = BeautifulSoup(lines, 'html.parser')
    
    data.append({})
    for row in [str(e).replace('</span>', '').split('>')[-1] for e in bs.find_all("span")]:
        if ':' in row:
            k, v = row.replace(': ', ':').split(':')
            data[i][k.strip()] = v.strip()
        else:
            data[i]['Год'] = row.strip()[-4:]
    
    data[i]['Строение'] = str(bs.find_all("h1")[0]).replace('</h1>', '').split(':')[-1].strip()
    data[i]['Ссылка на изображение'] = str(bs.find_all("img")[0])[:-2].split('=')[-1]
    
    address = str(bs.find_all("p")[0]).replace('</p>', '').replace('\n     ','').split('>')[-1].split('Индекс:')
    data[i]['Улица'] = address[0].replace('Улица: ', '').replace('Улица:', '')
    data[i]['Индекс'] = address[1].strip()

    for key in data[i]:
        if data[i][key].replace('.', '').isdigit():
            if '.' in data[i][key]:
                data[i][key] = float(data[i][key])
            else:
                data[i][key] = int(data[i][key])
        elif key == 'Парковка':
            data[i][key] = data[i][key] == 'Есть'
data[:3]

[{'Город': 'Варшава',
  'Этажи': 4,
  'Год': 1945,
  'Парковка': False,
  'Рейтинг': 2.6,
  'Просмотры': 66852,
  'Строение': 'Стадион 54',
  'Ссылка на изображение': '"https://upload.build.com/641819.jpeg"',
  'Улица': 'Юбилейная улица 4',
  'Индекс': 137266},
 {'Город': 'Таллин',
  'Этажи': 8,
  'Год': 1895,
  'Парковка': False,
  'Рейтинг': 4.6,
  'Просмотры': 84318,
  'Строение': 'Церковь 88',
  'Ссылка на изображение': '"https://upload.abc-build.it/996334.png"',
  'Улица': 'Речная улица 46',
  'Индекс': 256827},
 {'Город': 'Тирана',
  'Этажи': 10,
  'Год': 1673,
  'Парковка': False,
  'Рейтинг': 3.3,
  'Просмотры': 59832,
  'Строение': 'Подвал 6',
  'Ссылка на изображение': '"https://upload.buybuild.org/78549.webp"',
  'Улица': 'Киевская улица 50',
  'Индекс': 287783}]

In [2]:
import json


data = sorted(data, key=lambda x: x['Год'])
filtered_data = list(filter(lambda x: x['Рейтинг'] == 5, data))

with open('1_filtered.json', 'w', encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=True)
    
filtered_data[:3]

[{'Город': 'Загреб',
  'Этажи': 5,
  'Год': 1619,
  'Парковка': False,
  'Рейтинг': 5.0,
  'Просмотры': 72049,
  'Строение': 'Мост 95',
  'Ссылка на изображение': '"https://upload.build.com/8666.jpg"',
  'Улица': 'Варшавская улица 20',
  'Индекс': 227972},
 {'Город': 'Луго',
  'Этажи': 6,
  'Год': 1625,
  'Парковка': False,
  'Рейтинг': 5.0,
  'Просмотры': 69296,
  'Строение': 'Особняк 5',
  'Ссылка на изображение': '"https://upload.abc-build.it/532079.svg"',
  'Улица': 'Зимняя улица 62',
  'Индекс': 378903},
 {'Город': 'Лас-Росас',
  'Этажи': 7,
  'Год': 1649,
  'Парковка': False,
  'Рейтинг': 5.0,
  'Просмотры': 21923,
  'Строение': 'Тенета 6',
  'Ссылка на изображение': '"https://upload.build.com/319794.webp"',
  'Улица': 'Зимняя улица 53',
  'Индекс': 206590}]

In [3]:
import numpy as np

floor = [e['Этажи'] for e in data]
stats = {'Сумма': str(np.sum(floor)),
         'Минимум': str(np.min(floor)),
         'Максимум': str(np.max(floor)),
         'Среднее': str(np.mean(floor)),
         'Медиана': str(np.median(floor))}

with open("1_stats.json", "w", encoding="utf-8") as outfile: 
    json.dump(stats, outfile, ensure_ascii=True)

In [4]:
city = [e['Город'] for e in data]

freq = sorted(tuple(zip(*np.unique(city, return_counts=True))), key=lambda x: x[1], reverse=True)
freq = {e[0]: str(e[1]) for e in freq}
with open("1_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump(freq, outfile, ensure_ascii=True)

# Задание 2

In [5]:
fns = os.listdir('./2/zip_var_2')

data = []
for i, fn in enumerate(fns):
    with open(f'./2/zip_var_2/{fn}', encoding="utf-8") as f:
        lines = f.read()
    bs = BeautifulSoup(lines, 'html.parser')
    
    for product in bs.find_all("div", attrs={'class':"product-item"}):
        idx = len(data)
        data.append({})
        
        data[idx]['id'] = int(product.find_all('a')[0].attrs['data-id'])
        data[idx]['name'] = product.find_all('span')[0].text.strip()
        data[idx]['link'] = product.find_all('a')[1].attrs['href']
        data[idx]['price'] = int(product.find_all('price')[0].text.replace('₽', '').replace(' ', ''))
        data[idx]['bonus'] = int(product.find_all('strong')[0].text.split()[2])
        for category in product.find_all('li'):
            data[idx][category.attrs['type'].strip()] = category.text.strip()
        break
    
   
data[:3]

[{'id': 35593,
  'name': '6.7" Samsung 224GB',
  'link': '/product/35593',
  'price': 413700,
  'bonus': 4137,
  'sim': '2 SIM',
  'matrix': 'OLED'},
 {'id': 11194,
  'name': '5.2" OPPO 144GB',
  'link': '/product/11194',
  'price': 327740,
  'bonus': 3277,
  'processor': '2x3.4 ГГц',
  'ram': '8 GB',
  'sim': '3 SIM',
  'matrix': 'OLED',
  'resolution': '1280x1200',
  'camera': '92 MP',
  'acc': '2392 мА * ч'},
 {'id': 20938,
  'name': '7.3" Marvell 112GB',
  'link': '/product/20938',
  'price': 317261,
  'bonus': 3172,
  'processor': '2x4.6 ГГц',
  'ram': '9 GB',
  'sim': '1 SIM',
  'matrix': 'OLED',
  'resolution': '1366x1080',
  'acc': '4308 мА * ч'}]

In [6]:
data = sorted(data, key=lambda x: x['id'])
filtered_data = list(filter(lambda x: x['price'] < 100000, data))

with open('2_filtered.json', 'w', encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=True)
    
filtered_data[:3]

[{'id': 18503,
  'name': '7.4" Huawei P 240GB',
  'link': '/product/18503',
  'price': 44227,
  'bonus': 442,
  'ram': '8 GB',
  'acc': '3618 мА * ч'},
 {'id': 19413,
  'name': '5.3" iPhone 13 160GB',
  'link': '/product/19413',
  'price': 91001,
  'bonus': 910,
  'processor': '4x4.2 ГГц',
  'sim': '1 SIM',
  'matrix': 'IPS',
  'resolution': '1600x1080',
  'camera': '16 MP'},
 {'id': 19706,
  'name': '6.0" Qualcomm 128GB',
  'link': '/product/19706',
  'price': 84349,
  'bonus': 843,
  'processor': '4x2.6 ГГц',
  'ram': '7 GB',
  'sim': '2 SIM',
  'matrix': 'OLED',
  'camera': '20 MP',
  'acc': '3666 мА * ч'}]

In [7]:
bonus = [e['bonus'] for e in data]
stats = {'Сумма': str(np.sum(bonus)),
         'Минимум': str(np.min(bonus)),
         'Максимум': str(np.max(bonus)),
         'Среднее': str(np.mean(bonus)),
         'Медиана': str(np.median(bonus))}

with open("2_stats.json", "w", encoding="utf-8") as outfile: 
    json.dump(stats, outfile, ensure_ascii=True)

In [8]:
matrix = [e['matrix'] for e in data if 'matrix' in e]

freq = sorted(tuple(zip(*np.unique(matrix, return_counts=True))), key=lambda x: x[1], reverse=True)
freq = {e[0]: str(e[1]) for e in freq}
with open("2_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump(freq, outfile, ensure_ascii=True)

# Задание 3

In [9]:
fns = os.listdir('./3/zip_var_2')

data = []
for i, fn in enumerate(fns):
    with open(f'./3/zip_var_2/{fn}', encoding="utf-8") as f:
        lines = f.read()
    bs = BeautifulSoup(lines, 'xml')
    idx = len(data)
    data.append({})
    for item in [e for e in list(bs.find_all('star')[0].children) if e != '\n']:
        if item.name == 'radius':
            data[idx][item.name] = int(item.text.strip())
        elif item.name in ['rotation', 'age', 'distance', 'absolute-magnitude'] :
            data[idx][item.name] = float(item.text.strip().split()[0])
        else:
            data[idx][item.name] = item.text.strip()
   
data[:3]

[{'name': 'Солнце 4968',
  'constellation': 'Водолей',
  'spectral-class': 'F8Y',
  'radius': 235308875,
  'rotation': 432.44,
  'age': 1.18,
  'distance': 507925.62,
  'absolute-magnitude': 2.98},
 {'name': 'Антарес 2462',
  'constellation': 'Близнецы',
  'spectral-class': 'A2V',
  'radius': 803967522,
  'rotation': 569.55,
  'age': 0.07,
  'distance': 7464771.22,
  'absolute-magnitude': 7.59},
 {'name': 'Бетельгейзе 2041',
  'constellation': 'Дева',
  'spectral-class': 'U5H',
  'radius': 862039415,
  'rotation': 898.84,
  'age': 5.96,
  'distance': 6219264.48,
  'absolute-magnitude': 5.22}]

In [10]:
data = sorted(data, key=lambda x: x['age'])
filtered_data = list(filter(lambda x: x['distance'] < 500000, data))

with open('3_filtered.json', 'w', encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=True)
    
filtered_data[:3]

[{'name': 'Акрукс 4340',
  'constellation': 'Рак',
  'spectral-class': 'V9X',
  'radius': 417456698,
  'rotation': 374.06,
  'age': 0.08,
  'distance': 309329.91,
  'absolute-magnitude': 6.17},
 {'name': 'Антарес 1313',
  'constellation': 'Козерог',
  'spectral-class': 'M0G',
  'radius': 137317045,
  'rotation': 981.57,
  'age': 0.23,
  'distance': 352157.54,
  'absolute-magnitude': 1.47},
 {'name': 'Фомальгаут 2006',
  'constellation': 'Рыбы',
  'spectral-class': 'X7P',
  'radius': 29872177,
  'rotation': 333.15,
  'age': 0.4,
  'distance': 225975.39,
  'absolute-magnitude': 1.05}]

In [11]:
absolute_magnitude = [e['absolute-magnitude'] for e in data]
stats = {'Сумма': str(np.sum(absolute_magnitude)),
         'Минимум': str(np.min(absolute_magnitude)),
         'Максимум': str(np.max(absolute_magnitude)),
         'Среднее': str(np.mean(absolute_magnitude)),
         'Медиана': str(np.median(absolute_magnitude))}

with open("3_stats.json", "w", encoding="utf-8") as outfile: 
    json.dump(stats, outfile, ensure_ascii=True)

In [12]:
constellation = [e['constellation'] for e in data if 'constellation' in e]

freq = sorted(tuple(zip(*np.unique(constellation, return_counts=True))), key=lambda x: x[1], reverse=True)
freq = {e[0]: str(e[1]) for e in freq}
with open("3_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump(freq, outfile, ensure_ascii=True)

# Задание 4

In [13]:
fns = os.listdir('./4/zip_var_2')

data = []
for i, fn in enumerate(fns):
    with open(f'./4/zip_var_2/{fn}', encoding="utf-8") as f:
        lines = f.read()
    bs = BeautifulSoup(lines, 'xml')
    for clothing in bs.find_all('clothing'):
        idx = len(data)
        data.append({})
        for item in [e for e in list(bs.find_all('clothing')[0].children) if e != '\n']:
            if item.name in ['id', 'price', 'reviews']:
                data[idx][item.name] = int(item.text.strip())
            elif item.name == 'rating':
                data[idx][item.name] = float(item.text.strip())
            elif item.name in ['exclusive', 'sporty', 'new']:
                data[idx][item.name] = item.text.strip() in ['yes', '+']
            else:
                data[idx][item.name] = item.text.strip()
                
data[:3]

[{'id': 144380275,
  'name': 'Adidas Skirt L-8488',
  'category': 'Skirt',
  'size': 'M',
  'color': 'Синий',
  'material': 'Лен',
  'price': 561262,
  'rating': 1.6,
  'reviews': 741865,
  'new': False,
  'exclusive': True,
  'sporty': False},
 {'id': 144380275,
  'name': 'Adidas Skirt L-8488',
  'category': 'Skirt',
  'size': 'M',
  'color': 'Синий',
  'material': 'Лен',
  'price': 561262,
  'rating': 1.6,
  'reviews': 741865,
  'new': False,
  'exclusive': True,
  'sporty': False},
 {'id': 144380275,
  'name': 'Adidas Skirt L-8488',
  'category': 'Skirt',
  'size': 'M',
  'color': 'Синий',
  'material': 'Лен',
  'price': 561262,
  'rating': 1.6,
  'reviews': 741865,
  'new': False,
  'exclusive': True,
  'sporty': False}]

In [14]:
data = sorted(data, key=lambda x: x['id'])
filtered_data = list(filter(lambda x: x['rating'] > 4, data))

with open('4_filtered.json', 'w', encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=True)
    
filtered_data[:3]

[{'id': 115417128,
  'name': 'Under Armour Jacket S-6440',
  'category': 'Jacket',
  'size': 'L',
  'color': 'Синий',
  'material': 'Нейлон',
  'price': 437580,
  'rating': 4.71,
  'reviews': 957281,
  'sporty': False},
 {'id': 115417128,
  'name': 'Under Armour Jacket S-6440',
  'category': 'Jacket',
  'size': 'L',
  'color': 'Синий',
  'material': 'Нейлон',
  'price': 437580,
  'rating': 4.71,
  'reviews': 957281,
  'sporty': False},
 {'id': 115417128,
  'name': 'Under Armour Jacket S-6440',
  'category': 'Jacket',
  'size': 'L',
  'color': 'Синий',
  'material': 'Нейлон',
  'price': 437580,
  'rating': 4.71,
  'reviews': 957281,
  'sporty': False}]

In [15]:
reviews = [e['reviews'] for e in data]
stats = {'Сумма': str(np.sum(reviews)),
         'Минимум': str(np.min(reviews)),
         'Максимум': str(np.max(reviews)),
         'Среднее': str(np.mean(reviews)),
         'Медиана': str(np.median(reviews))}

with open("4_stats.json", "w", encoding="utf-8") as outfile: 
    json.dump(stats, outfile, ensure_ascii=True)

In [16]:
size = [e['size'] for e in data]

freq = sorted(tuple(zip(*np.unique(size, return_counts=True))), key=lambda x: x[1], reverse=True)
freq = {e[0]: str(e[1]) for e in freq}
with open("4_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump(freq, outfile, ensure_ascii=True)

# Задание 5 (несколько товаров)

In [17]:
fns = os.listdir('./intel-ekt_multi')

data = []
for i, fn in enumerate(fns):
    with open(f'./intel-ekt_multi/{fn}', encoding="utf-8") as f:
        lines = f.read()
    bs = BeautifulSoup(lines, 'html.parser')
    for product in bs.find_all('div', {"class": "main_list_product"}):
        idx = len(data)
        data.append({})
        
        data[idx]['id'] = int(product.find_all('div', {"class": "img"})[0].find_all('a')[0].attrs['href'].split('/')[-2])
        data[idx]['name'] = product.find_all('div', {"class": "name"})[0].text
        if len(product.find_all('div', {"class": "sprice"})) != 0:
            data[idx]['price'] = int(''.join(product.find_all('div', {"class": "sprice"})[0].text.split()[:-1]))
        for option in product.find_all('div', {"class": "options"})[0].text.split('; '):
            split = option.split(': ')
            if len(split) != 2:
                continue
            name, value = split
            
            if name in ['Количество ядер процессора', 'Разрешение основной камеры (точно)', 'Оперативная память', 'Емкость аккумулятора (точно)']:
                data[idx][name] = int(value.strip().split()[0].replace('.', ''))
            elif name in ['Диагональ экрана (точно)']:
                data[idx][name] = float(value.strip().split()[0])
            else:
                data[idx][name] = value.strip()
                
            data[idx][name] = value.strip().split()[0]
        
        if len(product.find_all('div', {"class": "available"})) != 0:
            data[idx]['isavailable'] = product.find_all('div', {"class": "available"})[0].text

data[:3]

[{'id': 139029,
  'name': 'Смартфон Xiaomi Redmi 9A 2/32GB Grey RU (Global Version)',
  'price': 5390,
  'Диагональ экрана (точно)': '6.53',
  'Количество ядер процессора': '8',
  'Разрешение основной камеры (точно)': '13',
  'Оперативная память': '2',
  'Емкость аккумулятора (точно)': '5000',
  'isavailable': 'Есть'},
 {'id': 145740,
  'name': 'Смартфон Xiaomi Redmi A2+ 3/64GB Black RU (Global Version)',
  'price': 5590,
  'isavailable': 'Есть'},
 {'id': 145824,
  'name': 'Смартфон Xiaomi Redmi A2+ 3/64GB Green RU (Global Version)',
  'price': 5990,
  'isavailable': 'Есть'}]

In [18]:
data = sorted(data, key=lambda x: x['id'])
filtered_data = list(filter(lambda x: 'price' in x and x['price'] > 30000, data))

with open('5_multi_filtered.json', 'w', encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=True)
    
filtered_data[:3]

[{'id': 143298,
  'name': 'Смартфон OnePlus Ace(10R) 12/256GB PGKM10 Light_Blue',
  'price': 32590,
  'isavailable': 'Есть'},
 {'id': 143299,
  'name': 'Смартфон OnePlus Ace(10R) 12/256GB PGKM10 Black',
  'price': 31590,
  'isavailable': 'Есть'},
 {'id': 143610,
  'name': 'Смартфон OnePlus Ace(10R) 12/512GB PGKM10 Black',
  'price': 33990,
  'isavailable': 'Есть'}]

In [19]:
capacity = [int(e['Емкость аккумулятора (точно)']) for e in data if 'Емкость аккумулятора (точно)' in e]
stats = {'Сумма': str(np.sum(capacity)),
         'Минимум': str(np.min(capacity)),
         'Максимум': str(np.max(capacity)),
         'Среднее': str(np.mean(capacity)),
         'Медиана': str(np.median(capacity))}

with open("5_multi_stats.json", "w", encoding="utf-8") as outfile: 
    json.dump(stats, outfile, ensure_ascii=True)

In [20]:
isavailable = [e['isavailable'] for e in data if 'isavailable' in e]

freq = sorted(tuple(zip(*np.unique(isavailable, return_counts=True))), key=lambda x: x[1], reverse=True)
freq = {e[0]: str(e[1]) for e in freq}
with open("5_multi_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump(freq, outfile, ensure_ascii=True)

# Задание 5 (один товаров)

In [22]:
fns = os.listdir('./intel-ekt_single')

data = []
for i, fn in enumerate(fns):
    with open(f'./intel-ekt_single/{fn}', encoding="utf-8") as f:
        lines = f.read()
    bs = BeautifulSoup(lines, 'html.parser').find_all('table', {'class': 'product'})[0]
    data.append({})
        
    data[i]['id'] = int(bs.find_all('td', {'class': 'idcode'})[0].text.strip().split()[1])
    data[i]['name'] = bs.find_all('img')[0].attrs['title']
    data[i]['price'] = int(''.join(bs.find_all('div', {'class': 'price'})[0].text.replace('\xa0', '').split(' ')[:-1])) 
    data[i]['isavailable'] = bs.find_all('div', {'class': 'available_on'})[0].text.split(': ')[-1]
    data[i]['delivery'] = [e.text.strip() for e in bs.find_all('div', {'class': 'item'})]
data[:3]

[{'id': 139029,
  'name': 'Смартфон Xiaomi Redmi 9A 2/32GB Grey RU (Global Version)',
  'price': 5390,
  'isavailable': 'ул. Ленина, 56 ул. Декабристов, 27',
  'delivery': ['Курьером: от 250 руб. Завтра', 'Самовывоз, бесплатно.']},
 {'id': 143839,
  'name': 'Смартфон Xiaomi Pocophone M5s NFC 4/128Gb Blue EU (Global Version)',
  'price': 13990,
  'isavailable': 'ул. Ленина, 56 ул. Декабристов, 27',
  'delivery': ['Курьером: от 250 руб. Завтра', 'Самовывоз, бесплатно.']},
 {'id': 145740,
  'name': 'Смартфон Xiaomi Redmi A2+ 3/64GB Black RU (Global Version)',
  'price': 5590,
  'isavailable': 'ул. Ленина, 56 ул. Декабристов, 27',
  'delivery': ['Курьером: от 250 руб. Завтра', 'Самовывоз, бесплатно.']}]

In [23]:
data = sorted(data, key=lambda x: x['id'])
filtered_data = list(filter(lambda x: 'price' in x and x['price'] > 10000, data))

with open('5_single_filtered.json', 'w', encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=True)
    
filtered_data[:3]

[{'id': 143512,
  'name': 'Смартфон Xiaomi Redmi 10 (2022) NFC 4/128Gb Blue EU (Global Version)',
  'price': 12990,
  'isavailable': 'ул. Ленина, 56 ул. Декабристов, 27',
  'delivery': ['Курьером: от 250 руб. Завтра', 'Самовывоз, бесплатно.']},
 {'id': 143839,
  'name': 'Смартфон Xiaomi Pocophone M5s NFC 4/128Gb Blue EU (Global Version)',
  'price': 13990,
  'isavailable': 'ул. Ленина, 56 ул. Декабристов, 27',
  'delivery': ['Курьером: от 250 руб. Завтра', 'Самовывоз, бесплатно.']},
 {'id': 145236,
  'name': 'Смартфон Xiaomi Redmi 12 4/128Gb Blue NFC RU (Global Version)',
  'price': 12990,
  'isavailable': 'ул. Ленина, 56',
  'delivery': ['Курьером: от 250 руб. Завтра',
   'Самовывоз (ул. Ленина, 56), бесплатно.']}]

In [24]:
price = [e['price'] for e in data]
stats = {'Сумма': str(np.sum(price)),
         'Минимум': str(np.min(price)),
         'Максимум': str(np.max(price)),
         'Среднее': str(np.mean(price)),
         'Медиана': str(np.median(price))}

with open("5_single_stats.json", "w", encoding="utf-8") as outfile: 
    json.dump(stats, outfile, ensure_ascii=True)

In [25]:
isavailable = [e['isavailable'] for e in data]

freq = sorted(tuple(zip(*np.unique(isavailable, return_counts=True))), key=lambda x: x[1], reverse=True)
freq = {e[0]: str(e[1]) for e in freq}
with open("5_single_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump(freq, outfile, ensure_ascii=True)